# Scrape Data From Reddit Pushshift

In [1]:
from Reddit_Loader import Loader
import datetime
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
# Set Start and End Date
START = "01/01/2020"
END = datetime.datetime.now().strftime("%d/%m/%Y") #today

In [13]:
# Load data from Pushshift and store it as a dataframe
l = Loader("gme", START, END, "wallstreetbets")
submissions = l.load_submissions()

Response cache key: 29983c8c40dd541667743b8ea5bd1315


Not all PushShift shards are active. Query results may be incomplete.


179167 result(s) available in Pushshift


Not all PushShift shards are active. Query results may be incomplete.


Total:: Success Rate: 80.71% - Requests: 2649 - Batches: 266 - Items Remaining: 354


Not all PushShift shards are active. Query results may be incomplete.


354 result(s) not found in Pushshift
Retrieved 178813 submissions from Pushshift.


In [15]:
# Save as csv
submissions.to_csv('./data/submissions.csv', encoding='utf-8', index=False)

In [14]:
# Save dataframe as pickle
submissions.to_pickle("./data/submissions.pkl")

In [91]:
submissions = pd.read_pickle("./data/submissions.pkl")

# Data Exploration

In [3]:
# All the available columns in the submissions dataframe
for element in submissions:
    print(element, end = ", ")

all_awardings, allow_live_comments, author, author_flair_css_class, author_flair_richtext, author_flair_text, author_flair_type, author_fullname, author_patreon_flair, author_premium, awarders, can_mod_post, contest_mode, created_utc, domain, full_link, gildings, id, is_crosspostable, is_meta, is_original_content, is_reddit_media_domain, is_robot_indexable, is_self, is_video, link_flair_background_color, link_flair_css_class, link_flair_richtext, link_flair_template_id, link_flair_text, link_flair_text_color, link_flair_type, locked, media_only, no_follow, num_comments, num_crossposts, over_18, parent_whitelist_status, permalink, pinned, pwls, retrieved_on, score, selftext, send_replies, spoiler, stickied, subreddit, subreddit_id, subreddit_subscribers, subreddit_type, suggested_sort, thumbnail, title, total_awards_received, treatment_tags, upvote_ratio, url, whitelist_status, wls, post_hint, preview, removed_by_category, is_gallery, thumbnail_height, thumbnail_width, url_overridden_by

In [4]:
l = ("all_awardings",
"allow_live_comments",
"author",
"author_flair_css_class",
"author_flair_richtext",
"author_flair_text",
"author_flair_type",
"author_fullname",
"author_patreon_flair",
"author_premium",
"awarders",
"can_mod_post",
"contest_mode",
"created_utc",
"domain",
"full_link",
"gildings",
"id",
"is_crosspostable",
"is_meta",
"is_original_content",
"is_reddit_media_domain",
"is_robot_indexable",
"is_self",
"is_video",
"link_flair_background_color",
"link_flair_css_class",
"link_flair_richtext",
"link_flair_template_id",
"link_flair_text",
"link_flair_text_color",
"link_flair_type",
"locked",
"media_only",
"no_follow",
"num_comments",
"num_crossposts",
"over_18",
"parent_whitelist_status",
"permalink",
"pinned",
"pwls",
"retrieved_on",
"score",
"selftext",
"send_replies",
"spoiler",
"stickied",
"subreddit",
"subreddit_id",
"subreddit_subscribers",
"subreddit_type",
"suggested_sort",
"thumbnail",
"title",
"total_awards_received",
"treatment_tags",
"upvote_ratio",
"url",
"whitelist_status",
"wls",
"post_hint",
"preview",
"removed_by_category",
"is_gallery",
"thumbnail_height",
"thumbnail_width",
"url_overridden_by_dest",
"media_metadata",
"media",
"media_embed",
"secure_media",
"secure_media_embed",
"author_flair_background_color",
"author_flair_text_color",
"edited",
"gallery_data",
"banned_by",
"author_cakeday",
"discussion_type",
"crosspost_parent",
"crosspost_parent_list",
"author_flair_template_id",
"author_is_blocked",
"is_created_from_ads_ui",
"steward_reports",
"gilded",
"distinguished",
"collections")

len(l)

89

In [5]:
# The number of Awards can be used as a proxy for the awards given
print(len(submissions["all_awardings"][44][0]["resized_icons"])) # We only count the unique awards given, not if an award has been given multiple times. May figure it out later.

# is_video can be useful to weed out posts, that do not have any content and only a video. Might still be able to perform sentiment analysis on the title.
# link_flair_text can be useful to classify content: Meme, Yolo, etc.
# subreddit_subscribers is a cool statistic
# thumbnail can be used to see if posts with a thumbnail have a bigger impact
# media refers to external links. Can probably remove all those entries

# To do:
# double check the upvote ratio, it seems to be wrong
# see how accurate removed_by_category is

5


In [92]:
# Drop unneeded columns
## allow_live_comments is Boolean
## author_flair_css_class shows the css class of the author_flair
## author_flair_richtext, author_flair_text, author_flair_type can be ignored, because the author can assign a flair to himself if he wants to
## author_fullname is just the id for the user
## author_patreon_flair, author_premium, domain, gildings, is_crosspostable, is_original_content, is_reddit_media_domain, is_robot_indexable,
## is_self, link_flair_background_color, link_flair_text_color, link_flair_css_class, link_flair_template_id, locked, media_only,
## no_follow, over_18, parent_whitelist_status, permalink, pwls, send_replies, spoiler, stickied, suggested_sort, subreddit_type, treatment_tags,
## whitelist_status, wls, preview, is_gallery, thumbnail_height, thumbnail_width, url_overridden_by_dest, media_metadata, secure_media,
## secure_media_embed, author_flair_background_color, author_flair_text_color, edited, gallery_data, banned_by do not seem to be helpful for sentiment analysis
## awarders is always empty
## can_mod_post, contest_mode, is_meta, pinned are always False
## link_flair_type, discussion_type, crosspost_parent, crosspost_parent_list, author_flair_template_id, author_is_blocked,
## is_created_from_ads_ui, steward_reports, gilded, distinguished, collections don't really say anything
## link_flair_richtext is a dictionary for for link_flair(which is not discarded)
## total_awards_received is similar to all_awardings, but not accurate
submissions = submissions.drop(columns = ["allow_live_comments", "author_flair_css_class", "author_flair_richtext", 
                            "author_flair_text", "author_flair_type", "author_fullname", "author_patreon_flair",
                            "author_premium", "domain", "awarders", "can_mod_post", "contest_mode",
                            "gildings", "is_crosspostable", "is_meta", "is_original_content", "is_reddit_media_domain",
                            "is_robot_indexable", "is_self", "link_flair_background_color", "link_flair_text_color",
                            "link_flair_type", "link_flair_css_class", "link_flair_richtext", "link_flair_template_id",
                            "locked", "media_only", "no_follow", "over_18", "parent_whitelist_status", "permalink",
                            "pinned", "pwls", "send_replies", "spoiler", "stickied", "subreddit_type", "suggested_sort",
                            "treatment_tags", "whitelist_status", "wls", "preview", "is_gallery", "thumbnail_height",
                            "thumbnail_width", "url_overridden_by_dest", "media_metadata", "secure_media", "secure_media_embed",
                            "author_flair_background_color", "author_flair_text_color", "edited", "gallery_data",
                            "banned_by", "discussion_type", "crosspost_parent", "crosspost_parent_list",
                            "author_flair_template_id", "author_is_blocked", "is_created_from_ads_ui",
                            "steward_reports", "gilded", "distinguished", "collections"])

In [93]:
print(submissions.columns)

Index(['all_awardings', 'author', 'created_utc', 'full_link', 'id', 'is_video',
       'link_flair_text', 'num_comments', 'num_crossposts', 'retrieved_on',
       'score', 'selftext', 'subreddit', 'subreddit_id',
       'subreddit_subscribers', 'thumbnail', 'title', 'total_awards_received',
       'upvote_ratio', 'url', 'post_hint', 'removed_by_category', 'media',
       'media_embed', 'author_cakeday'],
      dtype='object')


In [94]:
submissions["missing_content"] = False

In [95]:
submissions.loc[(submissions["selftext"] == "[removed]"), "missing_content"] = True

In [102]:
submissions.loc[(submissions["selftext"] == "[deleted]"), "missing_content"] = True

In [96]:
empty_content = []
for i, text in enumerate(submissions["selftext"]):
    if(len(str(text)) == 0):
        empty_content.append(i)

submissions.loc[empty_content, "missing_content"] = True


In [97]:
submissions["missing_content"].value_counts()

True     127402
False     51411
Name: missing_content, dtype: int64

In [106]:
title = "Lol I just joined today and bought GME at $90. What are yall doing?"
submissions[submissions["title"] == title][["selftext"]]

,selftext
24658,NaN


In [126]:
submissions.loc[(submissions["selftext"].isnull() == True), "missing_content"] = True

# Typical Powerlaw
Almost 80% of posts were deleted or contain no content!

In [129]:
ful_text = len(submissions[submissions["missing_content"] == False]["selftext"])
rem_text = len(submissions[submissions["missing_content"] == True]["selftext"])

# How many posts have the text not removed?
print("Posts with text:", ful_text)

# How many posts have the text "removed"?
print("Posts with removed text:", rem_text)

print("Proportion of posts with removed text:", round((rem_text)/(rem_text+ful_text),2))

Posts with text: 37591
Posts with removed text: 141222
Proportion of posts with removed text: 0.79


In [4]:
## Quite a lot of posts got removed. This is unfortunate. However, the title is still available. So I might still use the title.

# Who usually removes the post?
print("Removed by:\n", submissions[submissions["selftext"] == "[removed]"][["selftext", "removed_by_category"]].value_counts())
print("-----------------"*3)
# Were there any posts with a lot of comments that were removed?
print("Removed with x_comments:\n", submissions[submissions["selftext"] == "[removed]"][["selftext", "num_comments"]].value_counts())
print("-----------------"*3)
# Were there any posts with a high score that were removed?
print("Removed with x_comments:\n", submissions[submissions["selftext"] == "[removed]"][["selftext", "score"]].value_counts())

Removed by:
 selftext   removed_by_category
[removed]  moderator              63940
           reddit                  2198
           automod_filtered         620
           author                     1
dtype: int64
---------------------------------------------------
Removed with x_comments:
 selftext   num_comments
[removed]  0               52162
           2                8592
           1                1959
           4                 934
           3                 512
                           ...  
           186                 1
           178                 1
           176                 1
           174                 1
           64470               1
Length: 212, dtype: int64
---------------------------------------------------
Removed with x_comments:
 selftext   score
[removed]  1        60655
           2         1519
           0         1410
           3          459
           4          260
                    ...  
           180          1
           179 

In [5]:
# Put removed submissions into its own dataframe and then only use submissions that still exist.
submissions_removed = submissions[submissions["selftext"] == "[removed]"]
submissions = submissions[submissions["selftext"] != "[removed]"]

In [101]:
# Save submissions with flag to csv to do some manual analysis
submissions.to_csv('./data/test.csv', encoding='utf-8', index=False)